In [ ]:
import networkx as nx
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# for notebook
%matplotlib inline

In [ ]:
# grafico

def grafico_g(G, pos, rede):

    fig, ax = plt.subplots(1,1,figsize=(25,20))

    # draw edges
    nx.draw_networkx_edges(G, 
                           pos=pos, 
                           alpha=0.4, 
                           ax=ax)

    # draw nodes
    nodes = nx.draw_networkx_nodes(G,
                                   node_size=200,
                                   pos=pos,
                                   node_color='lightblue',  
                                   cmap=plt.cm.jet,
                                   ax=ax)

    # draw labels
    nx.draw_networkx_labels(G, 
                            pos=pos,
                            font_weight='bold',
                            font_color='black',
                            font_size=11, 
                            ax=ax)

    plt.title(rede)
    plt.axis("off")
    plt.show()

### Instancia

- **Stack Overflow Tag Network:** Network (links and nodes) of Stack Overflow tags based on Developer Stories.

- **Link:** https://www.kaggle.com/datasets/stackoverflow/stack-overflow-tag-network/data


In [ ]:
file_name = 'data/stack_network/stack_network_links.csv'

In [ ]:
df = pd.read_csv(file_name)

In [ ]:
df.head()

In [ ]:
SG = nx.Graph()

In [ ]:
with open(file_name) as f:
  next(f)
  SG = nx.parse_edgelist(f.readlines(), comments='/', delimiter=',', data=(('weight', float),))

In [ ]:
n = nx.number_of_nodes(SG)
m = nx.number_of_edges(SG)

print("#vertices =", n)
print("#arestas =", m)

print("rede eh conexa:", nx.is_connected(SG))

In [ ]:
list(SG.nodes())[:10]

In [ ]:
list(SG.edges.data('weight'))[:10]

In [ ]:
pos = nx.nx_pydot.graphviz_layout(SG)
grafico_g(SG, pos, "Stack Overflow Network")

### Grau

- O _degree_ de um nó é basicamente o número de arestas estão conectadas a $v$.

- O _degree centrality_ de um nó $v$ é a fração dos nós conectados a $v$.

- Os _degree centrality_ valores de G são normalizados pela divisão deles pelo máximo _degree_ da rede.

In [ ]:
# degree

deg = nx.degree(SG)

deg_list = sorted(dict(deg).items(), key=lambda node: node[1], reverse=True)[:10]

for t in deg_list:
    print(t)

In [ ]:
def grafico_g_medida(G, pos, medida, medida_nome):

    fig, ax = plt.subplots(1,1,figsize=(25,20))
    
    nodes = nx.draw_networkx_nodes(G, 
                                   pos, 
                                   node_size=200, 
                                   cmap=plt.cm.plasma, 
                                   node_color=list(medida.values()),
                                   nodelist=medida.keys(),
                                   ax=ax)
    
    #nodes.set_norm(mcolors.SymLogNorm(linthresh=0.01, linscale=1, base=10))
    
    labels = nx.draw_networkx_labels(G, pos)
    
    edges = nx.draw_networkx_edges(G, pos)

    plt.title(medida_nome)
    plt.colorbar(nodes)
    plt.axis('off')
    plt.show()

In [ ]:
# grafico da medida

pos = nx.nx_pydot.graphviz_layout(SG)
medida = dict(nx.degree(SG))
medida_nome = 'degree'
grafico_g_medida(SG, pos, medida, medida_nome)

In [ ]:
# degree with weight

degw = nx.degree(SG, weight='weight')

degw_list = sorted(dict(degw).items(), key=lambda node: node[1], reverse=True)[:10]

for t in degw_list:
    print(t)

In [ ]:
# degree_centrality with weight

#degcw = nx.degree_centrality(SG, weight='weight')

degw = nx.degree(SG,weight='weight')

max_degw = max(dict(degw).values())

degcw = {key:value/max_degw for key, value in dict(degw).items()}

degcw_list = sorted(dict(degcw).items(), key=lambda node: node[1], reverse=True)[:10]

for t in degcw_list:
    print(t)

### Excentricidade

- A _eccentricity_ de um nó $v$ é a distância máxima de $v$ para todos os outros nodes de $G$.

In [ ]:
# eccentricity

if nx.is_connected(SG) == True:
    
    ecc = nx.eccentricity(SG)

    ecc_list = sorted(dict(ecc).items(), key=lambda node: node[1], reverse=True)[:10]

    for t in ecc_list:
        print(t)
else:
    print("rede não eh conectada!")

In [ ]:
# eccentricity with weight

if nx.is_connected(SG) == True:

    eccw = nx.eccentricity(SG, weight='weight')

    eccw_list = sorted(dict(eccw).items(), key=lambda node: node[1], reverse=True)[:10]

    for t in eccw_list:
        print(t)
else:
    print("rede nao eh conectada!")

### Proximidade

- a _closeness centrality_ de um nó $u$ é a reciproca da média das _shortest path distance_ entre $u$ e todos os outros $n-1$ nós ligados a $u$.
$$
cloc(u) = \frac{n - 1}{\sum_{v=1}^{n-1} d(v, u)},
$$

- $d(v, u)$ é a _shortest-path distance_ entre $v$ e $u$.


In [ ]:
# closeness_centrality

cloc = nx.closeness_centrality(SG)

cloc_list = sorted(dict(cloc).items(), key=lambda node: node[1], reverse=True)[:10]

for t in cloc_list:
    print(t)

In [ ]:
# closeness_centrality with weight

#cloc = nx.closeness_centrality(SG, weight = 'weight')

g_distance_dict = {(e1, e2): 1 / weight for e1, e2, weight in SG.edges(data='weight')}
nx.set_edge_attributes(SG, g_distance_dict, 'distance')

cloc = nx.closeness_centrality(SG, distance='distance')

cloc_list = sorted(dict(cloc).items(), key=lambda node: node[1], reverse=True)[:10]

for t in cloc_list:
    print(t)

### Harmônica

- A _harmonic centrality_ de um nó $u$ é a soma da reciproca das _shortest path distances_ sobre todos os outros nós linkados a $u$.
$$
harc(u) = \sum_{v \neq u} \frac{1}{d(v, u)}
$$
onde $d(v, u)$ é a _shortest-path distance_ entre $v$ e $u$.

In [ ]:
# harmonic_centrality

harc = nx.harmonic_centrality(SG)

harc_list = sorted(dict(harc).items(), key=lambda node: node[1], reverse=True)[:10]

for t in harc_list:
    print(t)

In [ ]:
# harmonic_centrality with weight

#harcw = nx.harmonic_centrality(SG, weight='weight')

g_distance_dict = {(e1, e2): 1 / weight for e1, e2, weight in SG.edges(data='weight')}
nx.set_edge_attributes(SG, g_distance_dict, 'distance')

harcw = nx.harmonic_centrality(SG, distance='distance')

harcw_list = sorted(dict(harcw).items(), key=lambda node: node[1], reverse=True)[:10]

for t in harcw_list:
    print(t)

### Intermediação

- A _betweenness centrality_ de um nó $v$ é a soma da fração de todos as arestas dos _shortest paths_ que passam por $v$
$$
betc(v) =\sum_{s,t \in V} \frac{\sigma(s, t|v)}{\sigma(s, t)}
$$
onde $V$ é o conjunto dos nós, $\sigma(s, t)$ é o número de _shortest $(s, t)$-paths_, e $\sigma(s, t|v)$ é o número desses _paths_ que passam por $v$ onde v é diferente de $s$ ou $t$.

- Se $s = t$, $\sigma(s, t) = 1$, e se $v \in (s, t)$, $\sigma(s, t|v) = 0$.

In [ ]:
# betweenness_centrality

betc = nx.betweenness_centrality(SG, normalized = True)

betc = sorted(dict(betc).items(), key=lambda node: node[1], reverse=True)[:10]

for t in betc:
    print(t)

In [ ]:
# betweenness_centrality with weight

betcw = nx.betweenness_centrality(SG, weight = 'weight', normalized = True)

betcw_list = sorted(betcw.items(), key=lambda node: node[1], reverse=True)[:10]

for t in betcw_list:
    print(t)

### Autovetor

- A _eigenvector centrality_ é uma extensão da _degree centrality_.

- A _eigenvector centrality_ calcula a centralidade de um nó pela adição da centralidade de seus predecessores.

- A centralidade do nó $i$ é o $i$-th elemento de um _eigenvector_ associado a $i$ com o _eigenvalue_ $\lambda$ de módulo máximo e positivo.

In [ ]:
# eigenvector_centrality_numpy

eige = nx.eigenvector_centrality_numpy(SG)

eige_list = sorted(dict(eige).items(), key=lambda node: node[1], reverse=True)[:10]

for t in eige_list:
    print(t)

In [ ]:
# eigenvector_centrality with weight

eigew = nx.eigenvector_centrality_numpy(SG, weight='weight')

eigew_list = sorted(eigew.items(), key=lambda node: node[1], reverse=True)[:10]

for t in eigew_list:
    print(t)

### Katz

- _katz centrality_ calcula a centralidade de um nó baseada na centralidade de sua vizinhança. 

- _katz centrality_ é uma generalização da _eigenvector centrality_

In [ ]:
# katz

katz = nx.katz_centrality_numpy(SG, alpha=0.08, beta=1)

katz_list = sorted(dict(katz).items(), key=lambda node: node[1], reverse=True)[:10]

for t in katz_list:
    print(t)

In [ ]:
# katz with weight

katzw = nx.katz_centrality_numpy(SG, weight='weight', alpha=0.08, beta=1)

katzw_list = sorted(dict(katzw).items(), key=lambda node: node[1], reverse=True)[:10]

for t in katzw_list:
    print(t)

### Pagerank

- _PageRank_ calcula o _ranking_ dos nós grafo baseado na estrutura dos links de entrada.
 

In [ ]:
# pagerank

pagerank = nx.pagerank(SG, alpha=0.85)

pagerank_list = sorted(dict(pagerank).items(), key=lambda node: node[1], reverse=True)[:10]

for t in pagerank_list:
    print(t)

In [ ]:
# pagerank with weights

pagerankw = nx.pagerank(SG, alpha=0.85, weight='weight')

pagerankw_list = sorted(dict(pagerankw).items(), key=lambda node: node[1], reverse=True)[:10]

for t in pagerankw_list:
    print(t)

### Agrupamento

- Para gráficos não ponderados, o agrupamento de um nó $u$ é a fração de possíveis triângulos com esse nó
$$
c_u = \frac{2 T(u)}{deg(u)(deg(u)-1)},
$$
onde $T(u)$ é o número de triângulos em que o nó $u$ pertence e $deg(u)$ é o grau de $u$.

- Para grafos ponderados, existem várias definições para agrupamento, por exemplo, a média geométrica dos pesos das arestas dos subgrafos,
$$
c_u = \frac{1}{deg(u)(deg(u)-1))} \sum_{vw} (\hat{w}_{uv} \hat{w}_{uw} \hat{w}_{vw})^{1/3}.
$$
- O peso das arestas, $\hat{w}_{uv}$, são normalizados pelo máximo peso da rede, $\hat{w}_{uv} = w_{uv}/\max(w)$.

- O valor de $c_u$ é igual a $0$ se $deg(u) < 2$.

In [ ]:
# clustering

cluster = nx.clustering(SG)

cluster_list = sorted(dict(cluster).items(), key=lambda node: node[1], reverse=True)[:10]

for t in cluster_list:
    print(t)

In [ ]:
# clustering with weights

clusterw = nx.clustering(SG, weight='weight')

clusterw_list = sorted(dict(clusterw).items(), key=lambda node: node[1], reverse=True)[:10]

for t in clusterw_list:
    print(t)

### Agrupamento médio

- O valor do _clustering_ para um grafo G é a média,
$$
C = \frac{1}{n}\sum_{v \in G} c_v,
$$
onde $n$ é o número de nós em $G$.

In [ ]:
# average_clustering

print(nx.average_clustering(SG))

In [ ]:
# average_clustering with weight

print(nx.average_clustering(SG, weight='weight'))

### Componentes conectados do SG

In [ ]:
# componentes conexo

cc_len_list =[len(c) for c in sorted(nx.connected_components(SG), key=len, reverse=True)]

print("#componentes conexos:", len(cc_len_list))

print("tamanho de cada componete:", cc_len_list)

In [ ]:
# {} componetes conexos

set_cc = list(nx.connected_components(SG))

for t in set_cc:
    print(t)

In [ ]:
# subgrupo dos componentes conexos

subg_cc = [SG.subgraph(s).copy() for s in nx.connected_components(SG)]

for s in subg_cc:
    print(s)

In [ ]:
# componente 0

list(subg_cc[5].nodes())#[:10]

In [ ]:
list(subg_cc[0].edges.data('weight'))#[:10]

In [ ]:
# eccentricity with weight

# network
G = subg_cc[0]

eccw = nx.eccentricity(G, weight='weight')

eccw_list = sorted(dict(eccw).items(), key=lambda node: node[1], reverse=True)[:10]

for t in eccw_list:
        print(t)

In [ ]:
# closeness_centrality

# network
G = subg_cc[0]

cloc = nx.closeness_centrality(G)

cloc_list = sorted(dict(cloc).items(), key=lambda node: node[1], reverse=True)[:10]

for t in cloc_list:
    print(t)

### Atividade

- Calcular as medidas vista em sala de aula para a rede **subg_cc[0]** considerando os pesos da rede.

In [ ]:
# degree with weight

G = subg_cc[0]

deg = nx.degree(G, weight='weight')

deg_list = sorted(dict(deg).items(), key=lambda node: node[1], reverse=True)[:10]
for t in deg_list:
    print(t)